## A GeneratorContextManager Example

A short code example taken from an article on the O'Reilly website.

https://www.oreilly.com/learning/20-python-libraries-you-arent-using-but-should?imm_mid=0e7ab9&cmp=em-prog-na-na-newsltr_20160910

20 Python libraries you aren't using (but should)
Discover lesser-known Python libraries that are easy to install and use, cross-platform, and applicable to more than one domain.

By Caleb Hattingh
September 1, 2016

Hattingh's discussion of the code example is included in his article, but the following discussion tried to delve a bit deeper for those still learning.


First, a contextmanager is defined using the @contextmanager decorator.

In [1]:
from time import perf_counter
from array import array
from contextlib import contextmanager

@contextmanager  
def timing(label: str):
    t0 = perf_counter()  
    yield lambda: (label, t1 - t0)  
    t1 = perf_counter() 

The *timing* context manager features two calls to *time.perf_counter()* to calculate a time interval of short duration in fractional seconds.  

Under the covers, a ContextManager has an *enter()* method to build up the context and an *exit()* method to tear it down.  This is commonly understood when opening a file and having the ContextManager opens the file and closes it  automatically at the end.  With the @contextmanager the code up to the *yield* perpares the context and the code after it tears it down.

The *yield* indicates that *timing* is a a generator.  It yields a lambda function that returns a tuple with the assigned label for a given *timing* instance and the corresponding calculation of the time interval.  Note that the sequential nature of the code that places the calculation *t1 - t0* above the declaration of *t1* forces us to see that the yielded lambda is deferred and that we are dealing with a closure.  The lambda retains access to t0 and t1 after it is returned to the caller and when the lambda is called, as shown later on, both perf_counter values are available.

The rest of Hattingh's code uses the *timing* context manager to time two array operations sequentially and embeds them in another instance of *timer* to calculate a total.

In [2]:
with timing('Array tests') as total:  
    with timing('Array creation innermul') as inner:
        x = array('d', [0] * 1000000)  

    with timing('Array creation outermul') as outer:
        x = array('d', [0]) * 1000000 


print('Total [%s]: %.6f s' % total())
print('    Timing [%s]: %.6f s' % inner())
print('    Timing [%s]: %.6f s' % outer())


Total [Array tests]: 0.050259 s
    Timing [Array creation innermul]: 0.049044 s
    Timing [Array creation outermul]: 0.001179 s


Each of the instances of *timing* is associated with a variable name ("total", "inner", and "outer" following the *as* in each with statement) that is a reference to the yielded lambda closure; this is the means of access to the perf_counter calculation and the label as well.  

Each of these functions is static, returning the same values every time it is called since the perf_counter calls were done once per contextmanager.


In [3]:
outer()

('Array creation outermul', 0.0011788900010287762)

It is possible to inspect these as to how the closures are managed internally. The label, t0 and t1 are stored, not the result of the subtraction.

In [4]:
(outer.__closure__[0].cell_contents, outer.__closure__[1].cell_contents, outer.__closure__[2].cell_contents)

('Array creation outermul', 24640.371475214, 24640.372654104)